In [38]:
import numpy as np
import sympy as sp
import copy
from rdkit import Chem
import py3Dmol

In [39]:
mol_folder = './'
mol_fname = 'simple.mol2'
mol_filepath = mol_folder + mol_fname
mol = Chem.MolFromMol2File(mol_filepath)
mblock = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=500, height=250)
view.addModel(mblock, 'mol')
view.setStyle({'stick':{}})
view.zoomTo()
view.show()

[10:54:25] SIMPLE: Warning - no explicit hydrogens in mol2 file but needed for formal charge estimation.


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [40]:
coords_median = {}
torsional_bonds = {}
coords_rotation_dict = {}
distance_pairs = []
final_hubo = 0

In [41]:
n_atoms_mol = mol.GetNumAtoms()
conformers = mol.GetConformers()
conf = conformers[0]
coords_mol = {}
for i in range(n_atoms_mol):
    coords_mol[i] = [np.float16(0.0), np.float16(0.0), np.float16(0.0)]
    coords_mol[i][0] = np.float16(list(conf.GetAtomPosition(i))[0])
    coords_mol[i][1] = np.float16(list(conf.GetAtomPosition(i))[1])
    coords_mol[i][2] = np.float16(list(conf.GetAtomPosition(i))[2])
input_fname = mol_fname[:-5] + '_input.txt'
input_folder = './'
input_filepath = input_folder + input_fname
input_lines = open(input_filepath, 'r').readlines()
torsional_bonds_mol = eval(input_lines[0])
coords_rotation_mol = eval(input_lines[2])
distance_pairs_mol = eval(input_lines[3])

median_list_mol = []
for atoms in input_lines[1].split(','):
    median_list_mol.append(int(atoms))
for i, index in enumerate(median_list_mol):
    coords_median[i] = coords_mol[index]
for i in torsional_bonds_mol:
    torsional_bonds[i] = (median_list_mol.index(torsional_bonds_mol[i][0]), median_list_mol.index(torsional_bonds_mol[i][1]))
for atom in median_list_mol:
    if atom in coords_rotation_mol.keys():
        coords_rotation_dict[median_list_mol.index(atom)] = coords_rotation_mol[atom]
for x, y in distance_pairs_mol.items():
    u = median_list_mol.index(x)
    for i in y:
        v = median_list_mol.index(i)
        distance_pairs.append((u, v))
n_bonds = len(torsional_bonds)
x = sp.symbols(f'x(0:{2*n_bonds})', real=True)
coords_median_symb = copy.deepcopy(coords_median)

In [42]:
print(f'torsional_bonds: {torsional_bonds}')
print(f'coords_median: {coords_median}')
print(f'coords_rotation_dict: {coords_rotation_dict}')
print(f'distance_pairs: {distance_pairs}')

torsional_bonds: {0: (1, 2)}
coords_median: {0: [1.0, -0.5, 0.0], 1: [0.0, 0.0, 0.0], 2: [0.0, 1.0, 0.0], 3: [1.0, 1.5, 0.0]}
coords_rotation_dict: {3: [0]}
distance_pairs: [(0, 3), (1, 3)]


In [43]:
coords_median_symb

{0: [1.0, -0.5, 0.0],
 1: [0.0, 0.0, 0.0],
 2: [0.0, 1.0, 0.0],
 3: [1.0, 1.5, 0.0]}

In [73]:
for coords_median_key in coords_rotation_dict:
    final_rot_mat = sp.matrices.eye(4, 4)
    for bond_no in coords_rotation_dict[coords_median_key]:
        first_coords = coords_median[torsional_bonds[bond_no][0]]
        second_coords = coords_median[torsional_bonds[bond_no][1]]
        x_dash, y_dash, z_dash = first_coords[0], first_coords[1], first_coords[2]
        x_ddash, y_ddash, z_ddash = second_coords[0], second_coords[1], second_coords[2]
        dx = x_ddash - x_dash
        dy = y_ddash - y_dash
        dz = z_ddash - z_dash
        l_sq = np.float16(dx ** 2.0 + dy ** 2.0 + dz ** 2.0)
        l = np.sqrt(l_sq)
        c_theta = -1.0 + 0.03125*x[bond_no]
        s_theta = sp.expand_power_base(sp.sqrt(1 - c_theta*c_theta))
        rotation_matrix = sp.matrices.eye(4)
        rotation_matrix[0, 0] = (dx ** 2 + (dy ** 2 + dz ** 2) * c_theta) / l_sq
        rotation_matrix[0, 1] = (dx * dy * (1 - c_theta) - dz * l * s_theta) / l_sq
        rotation_matrix[0, 2] = (dx * dz * (1 - c_theta) + dy * l * s_theta) / l_sq
        rotation_matrix[0, 3] = ((x_dash * (dy ** 2 + dz ** 2) - dx * (y_dash * dy + z_dash * dz)) * (1 - c_theta) + (y_dash * dz - z_dash * dy) * l * s_theta) / l_sq
        rotation_matrix[1, 0] = (dx * dy * (1 - c_theta) + dz * l * s_theta) / l_sq
        rotation_matrix[1, 1] = (dy ** 2 + (dx ** 2 + dz ** 2) * c_theta) / l_sq
        rotation_matrix[1, 2] = (dy * dz * (1 - c_theta) - dx * l * s_theta) / l_sq
        rotation_matrix[1, 3] = ((y_dash * (dx ** 2 + dz ** 2) - dy * (x_dash * dx + z_dash * dz)) * (1 - c_theta) + (z_dash * dx - x_dash * dz) * l * s_theta) / l_sq
        rotation_matrix[2, 0] = (dx * dz * (1 - c_theta) - dy * l * s_theta) / l_sq
        rotation_matrix[2, 1] = (dy * dz * (1 - c_theta) + dx * l * s_theta) / l_sq
        rotation_matrix[2, 2] = (dz ** 2 + (dx ** 2 + dy ** 2) * c_theta) / l_sq
        rotation_matrix[2, 3] = ((z_dash * (dx ** 2 + dy ** 2) - dz * (x_dash * dx + y_dash * dy)) * (1 - c_theta) + (x_dash * dy - y_dash * dx) * l * s_theta) / l_sq
        final_rot_mat = final_rot_mat * rotation_matrix
    coord_vector = sp.matrices.ones(4, 1)
    coord_vector[0, 0] = copy.deepcopy(coords_median[coords_median_key][0])
    coord_vector[1, 0] = copy.deepcopy(coords_median[coords_median_key][1])
    coord_vector[2, 0] = copy.deepcopy(coords_median[coords_median_key][2])
    coord_rot_vector = (final_rot_mat * coord_vector).evalf(3).expand()
coords_median_symb[coords_median_key] = [(coord_rot_vector[0, 0]).expand(), (coord_rot_vector[1, 0]).expand(), (coord_rot_vector[2, 0]).expand()]
coords_median_symb

{0: [1.0, -0.5, 0.0],
 1: [0.0, 0.0, 0.0],
 2: [0.0, 1.0, 0.0],
 3: [0.0313*x0 - 1.0, 1.50, -0.25*(-0.0156*x0**2 + x0)**0.5]}

In [45]:
distance_pairs

[(0, 3), (1, 3)]

In [47]:
distance_constraint = 0
final_hubo = 0 
for pair in distance_pairs:
    x_coord, y_coord = coords_median_symb[pair[0]], coords_median_symb[pair[1]]
    ds_sq = 0.0
    for i in range(3):
        ds_sq += sp.expand((y_coord[i] - x_coord[i]) ** 2)
    distance_constraint += sp.expand(ds_sq)
final_hubo += sp.expand(distance_constraint)
final_hubo = -1.0*sp.expand_power_exp(final_hubo)
print(final_hubo)

-0.001953125*x0**2 + 0.1875*x0 - 0.125*(-0.0156*x0**2 + x0)**1.0 - 11.25


In [63]:
h = np.array([[0.0625]])
# [0.1875]
# [0]
J = np.array([[0.0]])
# [-0.001953125,    0.0         ]
# [0.0,             0.0    ]
H = np.hstack([h, J])
H
# [0.1875    , -0.00195312,  0.        ]
# [ 0.       ,  0.        ,  0.        ]

array([[0.0625, 0.    ]])

In [64]:
hamiltonian_data = {
  "data": [{"i": 0,"j": 0,"val": 0.0625},],
  "file_name": "simple_hamiltonian.json", # can be any short string
  "num_variables": 1, # number of rows
  "file_type": "hamiltonian" # defines the data type as hamiltonian
}
hamiltonian_data

{'data': [{'i': 0, 'j': 0, 'val': 0.0625}],
 'file_name': 'simple_hamiltonian.json',
 'num_variables': 1,
 'file_type': 'hamiltonian'}

In [65]:
import qci_client as qc
qci = qc.qci_client.QciClient(api_token="M3YYWg1ZaEuWkKI2eCIxFaFMoKeYem5m", url="https://api.qci-next.com")

In [66]:
response_json = qci.upload_file(hamiltonian_data)
job_params_instance = {"sampler_type": "eqc2", "n_samples": 1, "solution_type": "integer", "debug": True}
job_json = qci.build_job_body(
    job_type="sample-hamiltonian",
    job_params=job_params_instance,
    hamiltonian_file_id=response_json["file_id"]) 
job_response = qci.process_job(job_body=job_json, job_type="sample-hamiltonian")


Job submitted job_id='64e4962cf25d8eafde6e9e91'-: 2023/08/22 11:04:12
RUNNING: 2023/08/22 11:04:14
COMPLETED: 2023/08/22 11:04:32


In [68]:
results = job_response['results']
print(results['energies'][0])  # first entry, lowest energy
print(results['samples'][0]) # prints first solution array
sample = np.array(results["samples"][0]) # populates lowest energy sample into   a numpy array .

6.25
[63]


In [69]:
cos_theta = -1.0 + 0.03125*results["samples"][0][0]
cos_theta

0.96875

In [70]:
theta = np.arccos(cos_theta)
theta

0.2506556623361308

In [71]:
180*theta/np.pi

14.361511562916563

In [72]:
22/7

3.142857142857143